In [10]:
import numpy as np
from itertools import product
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
from lxml import etree
from lxml.builder import E
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
conf = Config('XDMFTest.yml')
sim = Simulator(conf)
sim.setup()
sim.get_fluxes()
sim.get_field()

In [13]:
grid = E.grid
doc = grid({'a1':'stuff',"a2":'diff stuff'})
print(etree.tostring(doc, pretty_print=True))

b'<grid a2="diff stuff" a1="stuff"/>\n'


In [38]:
def make_xml(sim):
    grid = E.Grid
    domain = E.Domain
    topo = E.Topology
    geo = E.Geometry
    ditem = E.DataItem
    attr = E.Attribute
    base = 'sim.hdf5:/sim_{}'.format(sim.id[0:10])
    dims = '{} {} {}'.format(sim.zsamps, sim.xsamps, sim.ysamps)
    doc = (
    E.Xdmf({'Version': '3.0'},
        domain(
            grid({'GridType': 'Uniform', 'Name': 'FullGrid'},
                topo({'TopologyType': '3DRectMesh'}),
                geo({'GeometryType': 'VXVYVZ'},
                   ditem(base+'/xcoords', {'Name': 'xcoords', 'Dimensions': str(sim.xsamps), 'NumberType': 'Float', 'Precision': '4', 'Format': 'HDF'}),
                   ditem(base+'/ycoords', {'Name': 'ycoords', 'Dimensions': str(sim.ysamps), 'NumberType': 'Float', 'Precision': '4', 'Format': 'HDF'}),
                   ditem(base+'/zcoords', {'Name': 'zcoords', 'Dimensions': str(sim.zsamps), 'NumberType': 'Float', 'Precision': '4', 'Format': 'HDF'})
                ),
                attr({'Name': 'Electric Field Components', 'AttributeType': 'Scalar', 'Center': 'Node'},
                    ditem(base+'/Ex', {'Dimensions': dims}),
                    ditem(base+'/Ey', {'Dimensions': dims}),
                    ditem(base+'/Ez', {'Dimensions': dims})
                )
            )
        )
    )
    )
    print(type(doc))
    tree = etree.ElementTree(doc)
    print(type(tree))
    print(etree.tostring(doc, pretty_print=True).decode())
make_xml(sim)

<class 'lxml.etree._Element'>
<class 'lxml.etree._ElementTree'>
<Xdmf Version="3.0">
  <Domain>
    <Grid Name="FullGrid" GridType="Uniform">
      <Topology TopologyType="3DRectMesh"/>
      <Geometry GeometryType="VXVYVZ">
        <DataItem Precision="4" Name="xcoords" Dimensions="5" NumberType="Float" Format="HDF">sim.hdf5:/sim_1b3d342aa8/xcoords</DataItem>
        <DataItem Precision="4" Name="ycoords" Dimensions="5" NumberType="Float" Format="HDF">sim.hdf5:/sim_1b3d342aa8/ycoords</DataItem>
        <DataItem Precision="4" Name="zcoords" Dimensions="10" NumberType="Float" Format="HDF">sim.hdf5:/sim_1b3d342aa8/zcoords</DataItem>
      </Geometry>
      <Attribute Name="Electric Field Components" AttributeType="Scalar" Center="Node">
        <DataItem Dimensions="10 5 5">sim.hdf5:/sim_1b3d342aa8/Ex</DataItem>
        <DataItem Dimensions="10 5 5">sim.hdf5:/sim_1b3d342aa8/Ey</DataItem>
        <DataItem Dimensions="10 5 5">sim.hdf5:/sim_1b3d342aa8/Ez</DataItem>
      </Attribute>
    

In [12]:
def old_make_xml(sim):
    root = et.Element('Xdmf')
    root.set('Version', '3.0')
    domain = et.SubElement(root, "Domain")
    grid = et.SubElement(domain, 'Grid')
    grid.set("GridType", "Uniform")
    grid.set("Name", "FullGrid")
    topo = et.SubElement(grid, "Topology")
    topo.set("TopologyType", "3DRectMesh")
    geo = et.SubElement(grid, "Geometry")
    geo.set("GeometryType", "VXVYVZ")
    points = et.SubElement(geo, "DataItem")
    gridpoints.set("Format", "XML")
    str_list = ["{} {} {}".format(*tup) for tup in product(x, y, z)]
    text = "\n".join(str_list)
    gridpoints.text = text
    ex = et.SubElement(grid, "Attribute")
    return root

In [ ]:
import xml.dom.minidom
root = make_xml(xvals, yvals, zvals)
xml_doc = xml.dom.minidom.parseString(et.tostring(root))
pretty_xml_as_string = xml_doc.toprettyxml()
print(pretty_xml_as_string)